In [1]:
import numpy as np
import pandas as pd
import torch, os, joblib
import torch.nn as nn
import torch.optim as optim
import optuna
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from torch.utils.data import DataLoader, TensorDataset
import torch, joblib, os
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset
## define the directory of current file 
directory = Path.cwd()

/softwarestack/miniconda3/envs/datascience/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load data
time_series_df = pd.read_pickle(os.path.join(directory, 'all_time_series_Consumption.pkl'))
time_series_df.set_index('Datetime', inplace=True)
labels_df_org = pd.read_pickle(os.path.join(directory, 'labels_df.pkl'))[['ID', 'Category']]

# Assign hourly datetime index (ensure proper format)
time_series_df.index = pd.date_range(start="2023-01-01", periods=time_series_df.shape[0], freq="H")

# Daily aggregation
daily_df = time_series_df.resample("D").agg(['mean', 'max', 'std', 'skew'])
daily_df.columns.set_names(["id", "stat"], inplace=True)
daily_df.columns = pd.MultiIndex.from_tuples([(str(id_), stat) for id_, stat in daily_df.columns])  # Convert ID to str

# Prepare label encoder
labels_df = labels_df_org.set_index('ID')
labels_df.index = labels_df.index.astype(str)  # Ensure matching with daily_df
label_encoder = LabelEncoder()
label_encoder.fit(labels_df['Category'])

# Sequence creation
sequence_length = 12
X = []
y = []

for id_ in labels_df.index:
    try:
        ts = daily_df.loc[:, id_]  # ✅ MultiIndex selection
    except KeyError:
        print(f"Skipping ID {id_} — not found in daily_df")
        continue
    
    ts_values = ts.values
    
    if ts_values.shape[0] < sequence_length:
        print(f"Skipping ID {id_} — not enough days for sequences")
        continue

    label = labels_df.loc[id_, 'Category']
    encoded_label = label_encoder.transform([label])[0]

    for i in range(len(ts_values) - sequence_length):
        X.append(ts_values[i:i+sequence_length])
        y.append(encoded_label)

X = np.array(X)
y = np.array(y)

print(f"✅ Created {len(X)} sequences across {len(set(labels_df.index))} IDs")

# Train/test split
if len(X) > 0:
    X_trainval, X_test, y_trainval, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y if len(set(y)) > 1 else None, random_state=42
    )
    print("✅ Train/test split complete")
else:
    raise ValueError("❌ No sequences were created. Check sequence length or data coverage.")


Using device: cuda


/tmp/ipykernel_2348681/1201258112.py:11: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  time_series_df.index = pd.date_range(start="2023-01-01", periods=time_series_df.shape[0], freq="H")


✅ Created 1882400 sequences across 1300 IDs
✅ Train/test split complete


In [16]:
##
# ---------------------
# 1. Define your model class (same as training)
# ---------------------
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_layers, dropout, bidirectional):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers,
                            batch_first=True, dropout=dropout, bidirectional=bidirectional)
        multiplier = 2 if bidirectional else 1
        self.fc = nn.Linear(hidden_dim * multiplier, output_dim)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.fc(out[:, -1, :])
#########
best_params = {  # 
    'hidden_dim': 254,
    'num_layers': 2,
    'dropout': 0.24846495923913972,
    'bidirectional': False,
    'input_dim': 4,  # daily features: mean, max, std, skew
    'output_dim': 5  # number of classes
}

model = LSTMModel(
    input_dim=best_params['input_dim'],
    hidden_dim=best_params['hidden_dim'],
    output_dim=best_params['output_dim'],
    num_layers=best_params['num_layers'],
    dropout=best_params['dropout'],
    bidirectional=best_params['bidirectional']
)

model.load_state_dict(torch.load (os.path.join(directory, 'best_lstm_model.pt')))
model.eval()

# Also reload your label encoder (or refit it if needed)

label_encoder = joblib.load((os.path.join(directory, 'label_encoder.pkl')))

# Use it for transforming or inverse transforming
label = label_encoder.inverse_transform([1]) 


In [17]:
# ---------------------
# 3. Prepare new raw input (e.g., hourly DataFrame)
# ---------------------
def prepare_sequences_from_raw(raw_df, sequence_length=12):
    raw_df.index = pd.to_datetime(raw_df.index)
    daily_df = raw_df.resample("D").agg(['mean', 'max', 'std', 'skew'])

    daily_df.columns.set_names(["id", "stat"], inplace=True)
    daily_df.columns = pd.MultiIndex.from_tuples([(str(id_), stat) for id_, stat in daily_df.columns])

    sequences = {}
    for id_ in raw_df.columns:
        try:
            ts = daily_df.loc[:, str(id_)]
        except KeyError:
            continue
        ts_values = ts.values
        if ts_values.shape[0] < sequence_length:
            continue
        sequences[id_] = [
            ts_values[i:i+sequence_length] for i in range(len(ts_values) - sequence_length + 1)
        ]
    return sequences

# ---------------------
# 4. Predict for each ID
# ---------------------
def predict_labels(sequences_dict):
    predictions = {}
    for id_, sequences in sequences_dict.items():
        X_tensor = torch.tensor(np.array(sequences), dtype=torch.float32)
        with torch.no_grad():
            logits = model(X_tensor)
            pred_indices = torch.argmax(logits, dim=1).cpu().numpy()
            pred_labels = label_encoder.inverse_transform(pred_indices)
            predictions[id_] = pred_labels
    return predictions

# ---------------------
# usage:
# ---------------------
# raw_time_series = pd.read_pickle("new_raw_hourly_data.pkl")  # shape: (timestamps x IDs)
#preds = predict_labels(prepare_sequences_from_raw(raw_time_series))
preds = predict_labels(X_test)

#for id_, labels in preds.items():
#     print(f"ID {id_}: Predicted labels for {len(labels)} sequences → {labels}")


AttributeError: 'numpy.ndarray' object has no attribute 'items'

In [ ]:
def create_sequences(data, labels, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(data) - seq_length):
        X_seq.append(data[i:i + seq_length])
        y_seq.append(labels[i])
    return np.array(X_seq), np.array(y_seq)

sequence_length = 12
X_full, y_full = create_sequences(time_series_scaled.T, labels_encoded, sequence_length)
X_trainval, X_test, y_trainval, y_test = train_test_split(X_full, y_full, test_size=0.2, stratify=y_full, random_state=42)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

def predict_from_tensor(model, X_array, device="cpu", batch_size=32):
    """
    Predicts from a numpy array using a trained model in batches.
    Avoids GPU memory overflow.
    """
    model.to(device)
    model.eval()

    dataset = TensorDataset(torch.tensor(X_array, dtype=torch.float32))
    loader = DataLoader(dataset, batch_size=batch_size)

    all_preds = []
    with torch.no_grad():
        for batch in loader:
            inputs = batch[0].to(device)
            logits = model(inputs)
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)

    return np.array(all_preds)
y_pred_indices = predict_from_tensor(model, X_test, device="cpu", batch_size=32)
y_pred_labels = label_encoder.inverse_transform(y_pred_indices)


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Convert y_test to label names
y_test_labels = label_encoder.inverse_transform(y_test)

# Create confusion matrix
cm = confusion_matrix(y_test_labels, y_pred_labels, labels=label_encoder.classes_)

# Plot
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=label_encoder.classes_,
            yticklabels=label_encoder.classes_)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Confusion Matrix (Test Set)")
plt.show()

# Optional: Classification report
print(classification_report(y_test_labels, y_pred_labels))
